### Import Libraries

In [1]:
import ee
import geemap
import geopandas as gpd
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from rasterio.merge import merge
import matplotlib.image as mpimg


### Initialize Earth Engine

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

### Read ROI

In [ ]:
# Load Mazandaran shapefile
shp_path = r"/shp_file/Caspian_Provinces.shp"
roi_gdf = gpd.read_file(shp_path)

# Convert to Earth Engine geometry
roi_ee = geemap.gdf_to_ee(roi_gdf, geodesic=False)

# Create a map and center it on the ROI
Map = geemap.Map(center=[37.25, 49.58], zoom=7)  
# Add base maps
Map.add_basemap('SATELLITE')
Map.add_basemap('ROADMAP')
# Add layer control
Map.addLayerControl()
# Add the ROI with styling
Map.addLayer(roi_ee,{'color': 'red', 'fillColor': '00000000'}, 'Mazandaran Province')
# Explicitly show the map
Map

### Indices for Landsat-8

In [6]:
def calculate_Indices_L8(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    
    evi = image.expression(
        '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)',
        {
            'NIR': image.select('B5'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }
    ).rename('EVI')
    
    savi = image.expression(
        '1.5 * (NIR - RED) / (NIR + RED + 1)',
        {
            'NIR': image.select('B5'),
            'RED': image.select('B4')
        }
    ).rename('SAVI')
    
    msavi = image.expression(
        '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED))) / 2',
        {
            'NIR': image.select('B5'),
            'RED': image.select('B4')
        }
    ).rename('MSAVI')
    
    combination = image.expression(
        '(NDVI + EVI + SAVI + MSAVI) / 4',
        {
            'NDVI': ndvi,
            'EVI': evi,
            'SAVI': savi,
            'MSAVI': msavi
        }
    ).rename('Combination')
    
    return image.addBands([ndvi, evi, savi, msavi, combination])


### Indices for Sentinel-2

In [7]:
def calculate_Indices_S2(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    evi = image.expression(
        '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }
    ).rename('EVI')
    
    savi = image.expression(
        '1.5 * (NIR - RED) / (NIR + RED + 1)',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ).rename('SAVI')
    
    msavi = image.expression(
        '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED))) / 2',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ).rename('MSAVI')
    
    combination = image.expression(
        '(NDVI + EVI + SAVI + MSAVI) / 4',
        {
            'NDVI': ndvi,
            'EVI': evi,
            'SAVI': savi,
            'MSAVI': msavi
        }
    ).rename('Combination')
    
    return image.addBands([ndvi, evi, savi, msavi, combination])

### Download scenes for Landsat-8 and Sentinel-2

In [ ]:
# Define date range and filter collections
start_date = '2024-04-01'
end_date = '2024-10-31'

# Landsat-8 collection
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA") \
    .filterBounds(roi_ee) \
    .filterDate(start_date, end_date) \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .map(calculate_Indices_L8)

# Sentinel-2 collection
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(roi_ee) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .map(calculate_Indices_S2)

# Calculate median for all indices and clip to ROI
landsat8_comb = landsat8.select(['NDVI', 'EVI', 'SAVI', 'MSAVI', 'Combination']).median().clip(roi_ee).toFloat()
sentinel2_comb = sentinel2.select(['NDVI', 'EVI', 'SAVI', 'MSAVI', 'Combination']).median().clip(roi_ee).toFloat()

# Export Landsat-8 IVI to Google Drive
landsat8_export_task = ee.batch.Export.image.toDrive(
    image=landsat8_comb,
    description='Landsat_2024_Caspian_Provinces',
    scale=30,
    crs='EPSG:4326',
    region=roi_ee.geometry(),
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# Export Sentinel-2 IVI to Google Drive
sentinel2_export_task = ee.batch.Export.image.toDrive(
    image=sentinel2_comb,
    description='Sentinel2_2024_Caspian_Provinces',
    scale=10,  # Sentinel-2 has 10m resolution
    crs='EPSG:4326',
    region=roi_ee.geometry(),
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# Start the export tasks
print("Starting export tasks to Google Drive...")
landsat8_export_task.start()
# sentinel2_export_task.start()

# Check task status
print("Landsat-8 task status:", landsat8_export_task.status())
# print("Sentinel-2 task status:", sentinel2_export_task.status())

Starting export tasks to Google Drive...
Landsat-8 task status: {'state': 'READY', 'description': 'Landsat_2024_Caspian_Provinces', 'priority': 100, 'creation_timestamp_ms': 1763973373782, 'update_timestamp_ms': 1763973373782, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'IKWCAVESLFVGIVJ4BE45F2HY', 'name': 'projects/60756559951/operations/IKWCAVESLFVGIVJ4BE45F2HY'}


### Visualization
After downlaoding the scenes, downlaoded from your Google Drive into images folder 

In [ ]:

# ---------------------------
# Paths - update these if needed
# ---------------------------
input_folder = r"images"
output_folder = r"output_png"
os.makedirs(output_folder, exist_ok=True)

# ---------------------------
# Index names (band order in the exported TIFF)
# ---------------------------
index_names = ["NDVI", "EVI", "SAVI", "MSAVI", "Combination"]

# ---------------------------
# Custom palette (user requested)
# ---------------------------
combo_colors = ["red", "orange", "yellow", "green"]
combo_cmap = LinearSegmentedColormap.from_list("combo", combo_colors)

# ---------------------------
# Visualization ranges - using data-driven ranges for better visualization
# ---------------------------
vis_ranges = {
    "NDVI":  "auto",
    "EVI":  "auto",
    "SAVI": "auto",  
    "MSAVI": "auto",  
    "Combination": "auto"  
}

# ---------------------------
# Function: mosaic several rasters into one
# ---------------------------
def mosaic_rasters(file_list, output_path):
    if len(file_list) == 0:
        raise ValueError("No files provided to mosaic: " + str(file_list))

    datasets = [rasterio.open(fp) for fp in file_list]
    mosaic_array, out_transform = merge(datasets)

    # Copy metadata from first dataset and update
    out_meta = datasets[0].meta.copy()
    out_meta.update({
        "height": mosaic_array.shape[1],
        "width": mosaic_array.shape[2],
        "transform": out_transform
    })

    with rasterio.open(output_path, "w", **out_meta) as dst:
        dst.write(mosaic_array)

    for ds in datasets:
        ds.close()

    print(f"[mosaic] Created {output_path} from {len(file_list)} files")
    return output_path

# ---------------------------
# Plotting function with improved range handling
# ---------------------------
def plot_indices_on_osm(tif_path, satellite_label):
    # Set global font to Times New Roman
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 11
    plt.rcParams['mathtext.fontset'] = 'stix'
    
    with rasterio.open(tif_path) as src:
        data = src.read()  # shape: (bands, rows, cols)
        bounds = src.bounds
        crs = src.crs

        # Safety: if fewer/more bands, warn
        if data.shape[0] < len(index_names):
            print(f"[warning] {tif_path} has {data.shape[0]} bands but expected {len(index_names)}.")
        bands_to_iterate = min(data.shape[0], len(index_names))

        for i in range(bands_to_iterate):
            name = index_names[i]
            band = data[i].astype(float)

            # Use original values for ALL indices without modifications
            display_band = np.where((band == 0) | np.isnan(band), np.nan, band)
            cmap_to_use = combo_cmap
            
            # Smart range selection based on data statistics
            valid_pixels = display_band[~np.isnan(display_band)]
            
            if valid_pixels.size == 0:
                print(f"[warning] No valid pixels for {satellite_label} {name} - skipping.")
                continue
                
            p2, p98 = np.percentile(valid_pixels, [2, 98])
            vmin, vmax = p2, p98
            print(f"{satellite_label} {name}: using percentiles {p2:.4f} to {p98:.4f}")

            # Create figure
            fig, ax = plt.subplots(figsize=(10, 8))

            # FIRST: Add basemap (if available) as bottom layer
            basemap_added = False
            try:
                # Set axis limits first
                ax.set_xlim([bounds.left, bounds.right])
                ax.set_ylim([bounds.bottom, bounds.top])
                
                # Try to get EPSG code
                if crs.is_epsg_code:
                    crs_epsg = crs.to_epsg()
                    ctx.add_basemap(ax, crs=crs_epsg, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.7)
                else:
                    ctx.add_basemap(ax, crs='EPSG:4326', source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.7)
                basemap_added = True
            except Exception as e:
                print(f"[warning] Basemap failed: {e} - proceeding without basemap")

            # SECOND: Plot your vegetation indices ON TOP
            img = ax.imshow(
                display_band,
                cmap=cmap_to_use,
                vmin=vmin,
                vmax=vmax,
                extent=[bounds.left, bounds.right, bounds.bottom, bounds.top],
                origin="upper",
                alpha=0.9 if basemap_added else 1.0
            )

            # Improved labels with degree symbols and better formatting
            ax.set_title(f"{satellite_label} — {name}", fontsize=14, fontweight='bold', pad=15)
            ax.set_xlabel("Longitude", fontsize=12, labelpad=10)
            ax.set_ylabel("Latitude", fontsize=12, labelpad=10)

            # Format tick labels with degree symbols
            ax.tick_params(axis='both', which='major', labelsize=10)
            
            # Format ticks with degree symbols and 2 decimal places
            ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f°'))
            ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f°'))
            
            # Ensure proper number of ticks
            ax.xaxis.set_major_locator(plt.MaxNLocator(6))
            ax.yaxis.set_major_locator(plt.MaxNLocator(6))

            # Make colorbar exactly same height as the image
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="3%", pad=0.1)

            cbar = plt.colorbar(img, cax=cax)
            cbar.ax.tick_params(labelsize=10)
            

            # Add grid for better readability
            ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

            # Save output PNG with high quality
            out_png = os.path.join(output_folder, f"{os.path.basename(tif_path).replace('.tif','')}_{name}.png")
            plt.savefig(out_png, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
            plt.close(fig)
            print(f"[saved] {out_png}")

# ---------------------------
# Gather files and create mosaics
# ---------------------------
landsat_files = []
sentinel_files = []

for fname in os.listdir(input_folder):
    if fname.lower().startswith("landsat") and fname.lower().endswith(".tif"):
        landsat_files.append(os.path.join(input_folder, fname))
    if fname.lower().startswith("sentinel") and fname.lower().endswith(".tif"):
        sentinel_files.append(os.path.join(input_folder, fname))

landsat_files.sort()
sentinel_files.sort()

if not landsat_files:
    raise FileNotFoundError("No Landsat files found in " + input_folder)
if not sentinel_files:
    raise FileNotFoundError("No Sentinel files found in " + input_folder)

landsat_mosaic_path = os.path.join(output_folder, "Landsat_Merged.tif")
sentinel_mosaic_path = os.path.join(output_folder, "Sentinel_Merged.tif")

mosaic_rasters(landsat_files, landsat_mosaic_path)
mosaic_rasters(sentinel_files, sentinel_mosaic_path)

# ---------------------------
# Run plotting for mosaics
# ---------------------------
plot_indices_on_osm(landsat_mosaic_path, "Landsat 8")
plot_indices_on_osm(sentinel_mosaic_path, "Sentinel 2")

print("All visualizations created.")

Landsat 8 NDVI: using percentiles 0.0609 to 0.7647
[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\Landsat_Merged_NDVI.png
Landsat 8 EVI: using percentiles 0.0632 to 0.8226
[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\Landsat_Merged_EVI.png
Landsat 8 SAVI: using percentiles 0.0259 to 0.3392
[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\Landsat_Merged_SAVI.png
Landsat 8 MSAVI: using percentiles 0.0348 to 0.5275
[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\Landsat_Merged_MSAVI.png
Landsat 8 Combination: using percentiles 0.0491 to 0.6094
[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\Landsat_Merged_Combination.png
Sentinel 2 NDVI: using percentiles 0.0363 to 0.8610
[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\Sentinel_Merged_NDVI.png
Sentinel 2 EVI: using percentiles 0.1237 to 2.7441
[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\Sentinel_Merged_EVI.png
Sentinel 2 SAVI: using percentil

### Generating comparison figures

In [ ]:
# ---------------------------------------------------
# Paths
# ---------------------------------------------------
folder = r"output_png"

# ---------------------------------------------------
# Image groups
# ---------------------------------------------------
comparison1 = [
    os.path.join(folder, "Landsat_Merged_NDVI.png"),
    os.path.join(folder, "Landsat_Merged_EVI.png"),
    os.path.join(folder, "Sentinel_Merged_NDVI.png"),
    os.path.join(folder, "Sentinel_Merged_EVI.png"),
]

comparison2 = [
    os.path.join(folder, "Landsat_Merged_SAVI.png"),
    os.path.join(folder, "Landsat_Merged_MSAVI.png"),
    os.path.join(folder, "Sentinel_Merged_SAVI.png"),
    os.path.join(folder, "Sentinel_Merged_MSAVI.png"),
]

# ---------------------------------------------------
# Function to make a 2×2 comparison figure
# ---------------------------------------------------
def make_comparison(img_paths, output_name):
    fig, axes = plt.subplots(2, 2, figsize=(12, 5))

    idx = 0
    for r in range(2):
        for c in range(2):
            img = mpimg.imread(img_paths[idx])
            axes[r, c].imshow(img)
            axes[r, c].axis("off")
            idx += 1

    # Reduce vertical and horizontal spacing
    plt.subplots_adjust(hspace=0.00, wspace=0.00)
    plt.savefig(output_name, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[saved] {output_name}")


# ---------------------------------------------------
# Create the two comparison figures
# ---------------------------------------------------
make_comparison(comparison1, os.path.join(folder, "comparison1.png"))
make_comparison(comparison2, os.path.join(folder, "comparison2.png"))

print("All comparison images created.")


[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\comparison1.png
[saved] D:\Books\Google Earth Engine\Projects\IVI\output_png\comparison2.png
All comparison images created.
